## Setup and Imports

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Create output directory
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

print("✓ Libraries imported successfully")
print(f"OpenCV Version: {cv2.__version__}")
print(f"NumPy Version: {np.__version__}")

## Part 1: Understanding Image Noise

### What is Image Noise?

Image noise is random variation in brightness or color information. It's particularly visible in:
- Low-light photography (high ISO settings)
- Long exposure shots
- Sensor limitations in smartphone cameras

### Types of Noise:
1. **Gaussian Noise**: Most common in digital cameras
2. **Salt-and-Pepper Noise**: Random black and white pixels
3. **Speckle Noise**: Multiplicative noise (common in radar/ultrasound)

In [ ]:
def create_sample_image(width=640, height=480):
    """Load cv.png or create a sample image if not found"""
    # Try to load cv.png
    image = cv2.imread('cv.png')
    
    if image is None:
        print("⚠ Warning: 'cv.png' not found. Creating sample image...")
        image = np.zeros((height, width, 3), dtype=np.uint8)
        
        # Add gradient background
        for i in range(height):
            image[i, :, 0] = int(255 * i / height)
            image[i, :, 1] = int(128 * (1 - i / height))
            image[i, :, 2] = int(200 * i / height)
        
        # Add scene content
        cv2.circle(image, (width//4, height//4), 60, (255, 200, 100), -1)
        cv2.rectangle(image, (width//2, height//3), (width//2 + 120, height//3 + 100), (100, 255, 200), -1)
        cv2.circle(image, (3*width//4, 3*height//4), 80, (200, 100, 255), -1)
        cv2.putText(image, 'Camera Image', (50, height-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    else:
        print("✓ Loaded 'cv.png' successfully")
    
    return image

def add_gaussian_noise(image, mean=0, std=25):
    """Add Gaussian noise to simulate low-light conditions"""
    noise = np.random.normal(mean, std, image.shape)
    noisy_image = np.clip(image.astype(np.float32) + noise, 0, 255).astype(np.uint8)
    return noisy_image

# Create clean image
clean_image = create_sample_image()

# Add noise (simulating low-light smartphone photo)
noisy_image = add_gaussian_noise(clean_image, mean=0, std=25)

# Display
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(cv2.cvtColor(clean_image, cv2.COLOR_BGR2RGB))
axes[0].set_title('Clean Image (Ideal Lighting)', fontsize=12)
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(noisy_image, cv2.COLOR_BGR2RGB))
axes[1].set_title('Noisy Image (Low-Light Conditions)', fontsize=12)
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("✓ Sample images created")

## Part 2: Understanding Convolution

### What is Convolution?

Convolution is a mathematical operation that slides a **kernel** (small matrix) over an image and computes weighted sums.

**Process:**
1. Place kernel over a region of pixels
2. Multiply each pixel by corresponding kernel weight
3. Sum all products to get output pixel value
4. Slide kernel to next position

### Convolution Formula:

$$ G(x, y) = \sum_{i=-k}^{k} \sum_{j=-k}^{k} K(i, j) \cdot F(x+i, y+j) $$

Where:
- $G(x,y)$ = Output pixel
- $F(x,y)$ = Input image
- $K(i,j)$ = Kernel weights

In [ ]:
# Visualize different convolution kernels
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
fig.suptitle('Common Convolution Kernels for Denoising', fontsize=14, fontweight='bold')

# 1. Average (Box) Filter
average_kernel = np.ones((5, 5)) / 25
im1 = axes[0].imshow(average_kernel, cmap='hot', interpolation='nearest')
axes[0].set_title('Average Filter\n(Equal Weights)', fontsize=11)
for i in range(5):
    for j in range(5):
        axes[0].text(j, i, f'{average_kernel[i,j]:.2f}', ha='center', va='center', color='white', fontsize=8)
plt.colorbar(im1, ax=axes[0], fraction=0.046)

# 2. Gaussian Filter
gaussian_kernel = cv2.getGaussianKernel(5, 1) @ cv2.getGaussianKernel(5, 1).T
im2 = axes[1].imshow(gaussian_kernel, cmap='hot', interpolation='nearest')
axes[1].set_title('Gaussian Filter\n(Weighted by Distance)', fontsize=11)
for i in range(5):
    for j in range(5):
        axes[1].text(j, i, f'{gaussian_kernel[i,j]:.2f}', ha='center', va='center', color='white', fontsize=8)
plt.colorbar(im2, ax=axes[1], fraction=0.046)

# 3. Identity (No Filter)
identity_kernel = np.zeros((5, 5))
identity_kernel[2, 2] = 1
im3 = axes[2].imshow(identity_kernel, cmap='hot', interpolation='nearest')
axes[2].set_title('Identity Filter\n(No Change)', fontsize=11)
for i in range(5):
    for j in range(5):
        axes[2].text(j, i, f'{identity_kernel[i,j]:.0f}', ha='center', va='center', 
                    color='white' if identity_kernel[i,j] > 0 else 'black', fontsize=8)
plt.colorbar(im3, ax=axes[2], fraction=0.046)

plt.tight_layout()
plt.show()

print("Key Observations:")
print("• Average Filter: All weights are equal (1/25 = 0.04)")
print("• Gaussian Filter: Center has highest weight, decreases with distance")
print("• Identity Filter: Only center pixel contributes (no filtering)")

## Part 3: Applying Linear Filters

### Filter Types:

1. **Average Filter (Box Filter)**
   - Simple arithmetic mean of neighborhood
   - Fast to compute
   - Treats all neighbors equally

2. **Gaussian Filter**
   - Weighted average based on Gaussian distribution
   - Better preserves edges
   - More "natural" smoothing

### Gaussian Function:

$$ G(x, y) = \frac{1}{2\pi\sigma^2} e^{-\frac{x^2 + y^2}{2\sigma^2}} $$

Where $\sigma$ controls the "spread" of the blur.

In [ ]:
# Apply different filters
average_filtered = cv2.blur(noisy_image, (5, 5))
gaussian_filtered = cv2.GaussianBlur(noisy_image, (5, 5), 1.5)

# Calculate PSNR (Peak Signal-to-Noise Ratio)
psnr_noisy = cv2.PSNR(clean_image, noisy_image)
psnr_average = cv2.PSNR(clean_image, average_filtered)
psnr_gaussian = cv2.PSNR(clean_image, gaussian_filtered)

# Display results
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
fig.suptitle('Noise Reduction Comparison', fontsize=16, fontweight='bold')

axes[0, 0].imshow(cv2.cvtColor(clean_image, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original (Clean)', fontsize=12)
axes[0, 0].axis('off')

axes[0, 1].imshow(cv2.cvtColor(noisy_image, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title(f'Noisy (Low-Light)\nPSNR: {psnr_noisy:.2f} dB', fontsize=12)
axes[0, 1].axis('off')

axes[1, 0].imshow(cv2.cvtColor(average_filtered, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title(f'Average Filter (5x5)\nPSNR: {psnr_average:.2f} dB\nImprovement: {psnr_average-psnr_noisy:.2f} dB', fontsize=12)
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(gaussian_filtered, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title(f'Gaussian Filter (5x5, σ=1.5)\nPSNR: {psnr_gaussian:.2f} dB\nImprovement: {psnr_gaussian-psnr_noisy:.2f} dB', fontsize=12)
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("\n📊 PERFORMANCE METRICS:")
print("=" * 50)
print(f"Noisy Image PSNR: {psnr_noisy:.2f} dB")
print(f"Average Filter PSNR: {psnr_average:.2f} dB (↑{psnr_average-psnr_noisy:.2f} dB)")
print(f"Gaussian Filter PSNR: {psnr_gaussian:.2f} dB (↑{psnr_gaussian-psnr_noisy:.2f} dB)")
print("\nNote: Higher PSNR = Better quality (typically > 30 dB is good)")

## Part 4: Trade-offs Analysis

### Smoothing vs. Detail Preservation

There's always a trade-off:
- **More smoothing** = Less noise BUT more blur
- **Less smoothing** = More details BUT more noise

Let's examine this by varying kernel size:

In [ ]:
# Compare different kernel sizes
kernel_sizes = [3, 5, 7, 9, 11]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Effect of Kernel Size on Denoising\n(Larger kernel = More smoothing)', 
             fontsize=14, fontweight='bold')

axes[0, 0].imshow(cv2.cvtColor(noisy_image, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Noisy Original', fontsize=11)
axes[0, 0].axis('off')

for idx, k_size in enumerate(kernel_sizes):
    row = (idx + 1) // 3
    col = (idx + 1) % 3
    
    filtered = cv2.GaussianBlur(noisy_image, (k_size, k_size), 0)
    psnr = cv2.PSNR(clean_image, filtered)
    
    axes[row, col].imshow(cv2.cvtColor(filtered, cv2.COLOR_BGR2RGB))
    axes[row, col].set_title(f'{k_size}x{k_size} Gaussian\nPSNR: {psnr:.2f} dB', fontsize=11)
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

print("Observations:")
print("• Small kernels (3x3, 5x5): Preserve details, moderate noise reduction")
print("• Large kernels (9x9, 11x11): Strong noise reduction, loss of fine details")
print("• Optimal choice depends on application requirements")

## Part 5: Advanced Filter Comparison

Let's compare linear filters with some advanced techniques used in modern cameras:

In [ ]:
# Compare different filter types
filters = {
    'Noisy Original': noisy_image,
    'Average (5x5)': cv2.blur(noisy_image, (5, 5)),
    'Gaussian (5x5)': cv2.GaussianBlur(noisy_image, (5, 5), 1.5),
    'Median (5x5)': cv2.medianBlur(noisy_image, 5),
    'Bilateral\n(Edge-Preserving)': cv2.bilateralFilter(noisy_image, 9, 75, 75)
}

fig, axes = plt.subplots(2, 3, figsize=(16, 11))
fig.suptitle('Comparison of Different Filtering Techniques', fontsize=16, fontweight='bold')

# Add clean reference
axes[0, 0].imshow(cv2.cvtColor(clean_image, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Clean Reference', fontsize=11)
axes[0, 0].axis('off')

for idx, (name, filtered) in enumerate(filters.items()):
    row = (idx + 1) // 3
    col = (idx + 1) % 3
    
    psnr = cv2.PSNR(clean_image, filtered)
    
    axes[row, col].imshow(cv2.cvtColor(filtered, cv2.COLOR_BGR2RGB))
    axes[row, col].set_title(f'{name}\nPSNR: {psnr:.2f} dB', fontsize=11)
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

print("\n🔍 FILTER COMPARISON:")
print("=" * 60)
print("Average Filter:")
print("  • Linear, simple arithmetic mean")
print("  • Fast, but can blur edges significantly")
print("")
print("Gaussian Filter:")
print("  • Linear, weighted by distance")
print("  • Better than average, more natural smoothing")
print("")
print("Median Filter:")
print("  • Non-linear, takes median value")
print("  • Excellent for salt-and-pepper noise")
print("")
print("Bilateral Filter:")
print("  • Edge-preserving, considers both space and intensity")
print("  • Used in smartphone cameras (Samsung, Apple)")
print("  • Slower but better quality")

## Part 6: Real-World Industry Applications

### 📱 Smartphone Cameras (Samsung Galaxy, Apple iPhone)

**Multi-Frame Noise Reduction:**
1. Capture multiple frames rapidly
2. Align frames (compensate for hand shake)
3. Apply temporal filtering (average across frames)
4. Apply spatial filters (Gaussian, bilateral)
5. Enhance details with sharpening

**Night Mode Pipeline:**
- Captures 6-9 frames over 1-3 seconds
- Uses optical image stabilization (OIS)
- Combines with AI-based enhancement
- Result: Clean, bright images even in very low light

### 🎥 CCTV Footage Enhancement

**Challenges:**
- Low-light conditions
- Limited computational resources
- Real-time requirements

**Solutions:**
- Fast linear filters (average, Gaussian)
- Adaptive filtering based on scene brightness
- Hardware acceleration (GPU)

### 🏥 Medical Imaging

- X-ray denoising
- MRI image enhancement
- Critical: Must preserve diagnostic details

## Part 7: Discussion Points & Key Takeaways

### 1️⃣ How Convolution is Used in Denoising:

- **Averaging neighboring pixels** reduces random noise
- **Kernel weights** determine how much each neighbor contributes
- **Gaussian kernel** gives more weight to nearby pixels (natural falloff)
- **Larger kernels** smooth over larger areas (more denoising, more blur)

### 2️⃣ Trade-offs: Smoothing vs. Detail Preservation:

| Aspect | More Smoothing | Less Smoothing |
|--------|----------------|----------------|
| Noise Reduction | ✓ Excellent | ✗ Poor |
| Detail Preservation | ✗ Lost | ✓ Retained |
| Edge Sharpness | ✗ Blurred | ✓ Sharp |
| PSNR | Higher initially, then drops | Moderate |
| Kernel Size | Large (9x9, 11x11) | Small (3x3, 5x5) |

**Best Practice:** Choose kernel size based on:
- Noise level in image
- Importance of fine details
- Computational constraints
- Application requirements

### 3️⃣ Real-World Relevance:

**Why This Matters:**
- Billions of smartphone photos taken daily
- Security and surveillance rely on clear footage
- Medical diagnoses depend on clean images
- Autonomous vehicles need clean sensor data

**Industry Trends:**
- Moving beyond simple linear filters
- AI/ML-based denoising (deep learning)
- Combination of multiple techniques
- Hardware acceleration (neural processing units)

**However, linear filters remain important because:**
- Fast and efficient
- Predictable behavior
- Low computational cost
- Foundation for understanding advanced techniques

## Part 8: Interactive Experimentation

Try modifying these parameters to see the effects:

In [ ]:
# EXPERIMENT: Try different values!
NOISE_LEVEL = 30          # Try: 10, 20, 30, 40
KERNEL_SIZE = 7           # Try: 3, 5, 7, 9, 11 (must be odd)
GAUSSIAN_SIGMA = 2.0      # Try: 0.5, 1.0, 2.0, 3.0

# Create noisy image with your noise level
experimental_noisy = add_gaussian_noise(clean_image, mean=0, std=NOISE_LEVEL)

# Apply filters with your parameters
exp_average = cv2.blur(experimental_noisy, (KERNEL_SIZE, KERNEL_SIZE))
exp_gaussian = cv2.GaussianBlur(experimental_noisy, (KERNEL_SIZE, KERNEL_SIZE), GAUSSIAN_SIGMA)

# Calculate metrics
psnr_noisy_exp = cv2.PSNR(clean_image, experimental_noisy)
psnr_avg_exp = cv2.PSNR(clean_image, exp_average)
psnr_gauss_exp = cv2.PSNR(clean_image, exp_gaussian)

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle(f'Experiment: Noise={NOISE_LEVEL}, Kernel={KERNEL_SIZE}x{KERNEL_SIZE}, σ={GAUSSIAN_SIGMA}', 
             fontsize=13, fontweight='bold')

axes[0].imshow(cv2.cvtColor(experimental_noisy, cv2.COLOR_BGR2RGB))
axes[0].set_title(f'Noisy\nPSNR: {psnr_noisy_exp:.2f} dB', fontsize=11)
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(exp_average, cv2.COLOR_BGR2RGB))
axes[1].set_title(f'Average Filter\nPSNR: {psnr_avg_exp:.2f} dB\n(↑{psnr_avg_exp-psnr_noisy_exp:.2f} dB)', fontsize=11)
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(exp_gaussian, cv2.COLOR_BGR2RGB))
axes[2].set_title(f'Gaussian Filter\nPSNR: {psnr_gauss_exp:.2f} dB\n(↑{psnr_gauss_exp-psnr_noisy_exp:.2f} dB)', fontsize=11)
axes[2].axis('off')

plt.tight_layout()
plt.show()

print(f"\n📊 Results with your parameters:")
print(f"Noise Level: {NOISE_LEVEL} | Kernel Size: {KERNEL_SIZE}x{KERNEL_SIZE} | Sigma: {GAUSSIAN_SIGMA}")
print(f"Average Filter Improvement: {psnr_avg_exp-psnr_noisy_exp:.2f} dB")
print(f"Gaussian Filter Improvement: {psnr_gauss_exp-psnr_noisy_exp:.2f} dB")
print(f"\nBetter Filter: {'Gaussian' if psnr_gauss_exp > psnr_avg_exp else 'Average'} (by {abs(psnr_gauss_exp-psnr_avg_exp):.2f} dB)")

## Summary

### ✅ What We Learned:

1. **Image noise** is common in low-light photography
2. **Convolution** is the mathematical foundation of filtering
3. **Linear filters** (average, Gaussian) reduce noise through local averaging
4. **Trade-offs exist** between noise reduction and detail preservation
5. **Modern cameras** use sophisticated multi-frame and AI techniques
6. **PSNR** is a useful metric for measuring image quality

### 🎯 Key Recommendations:

- For general denoising: **Gaussian filter (5x5, σ=1-2)**
- For edge preservation: **Bilateral filter**
- For real-time processing: **Average filter**
- For best results: **Combine multiple techniques**

### 🚀 Next Steps:

- Explore non-linear filters (median, bilateral)
- Study multi-frame denoising
- Learn about deep learning-based denoising
- Implement custom filters for specific applications

---

**Industry Applications:**
- 📱 Smartphone computational photography
- 🎥 Video surveillance enhancement
- 🏥 Medical image processing
- 🛰️ Satellite imagery
- 🚗 Autonomous vehicle perception